In [16]:
from pulp import *

prob = LpProblem("Maksonsel Distribution Problem", LpMinimize)

#define vars
plants = ['P1', 'P2']
warehouses = ['W1', 'W2']
retail_outlets = ['R1', 'R2', 'R3']

#defins constraints
costs_pw = {
    'P1': {'W1': 425, 'W2': 560},
    'P2': {'W1': 510, 'W2': 600}
}
capacity_pw = {
    'P1': {'W1': 125, 'W2': 150},
    'P2': {'W1': 175, 'W2': 200}
}
output = {'P1': 200, 'P2': 300}
costs_wr = {
    'W1': {'R1': 470, 'R2': 505, 'R3': 490},
    'W2': {'R1': 390, 'R2': 410, 'R3': 440}
}
capacity_wr = {
    'W1': {'R1': 100, 'R2': 150, 'R3': 100},
    'W2': {'R1': 125, 'R2': 150, 'R3': 75}
}
demand = {'R1': 150, 'R2': 200, 'R3': 150}

#powerplant to warehouse
x = LpVariable.dicts("Flow_P_W", (plants, warehouses), lowBound=0, cat='Integer')

#warehouse to retail
y = LpVariable.dicts("Flow_W_R", (warehouses, retail_outlets), lowBound=0, cat='Integer')

#adding constraints to prob
prob += lpSum([costs_pw[i][j] * x[i][j] for i in plants for j in warehouses]) + \
        lpSum([costs_wr[j][k] * y[j][k] for j in warehouses for k in retail_outlets]), "Total Shipping Cost"

for i in plants:
    prob += lpSum([x[i][j] for j in warehouses]) <= output[i], f"Plant {i} Output"

for i in plants:
    for j in warehouses:
        prob += x[i][j] <= capacity_pw[i][j], f"P{i}_W{j}_Capacity"

for j in warehouses:
    prob += lpSum([x[i][j] for i in plants]) == lpSum([y[j][k] for k in retail_outlets]), f"Warehouse {j} Flow Conservation"

for j in warehouses:
    for k in retail_outlets:
        prob += y[j][k] <= capacity_wr[j][k], f"W{j}_R{k}_Capacity"

for k in retail_outlets:
    prob += lpSum([y[j][k] for j in warehouses]) == demand[k], f"Retail Outlet {k} Demand"

print("-------Linear Programming Model-------\n")
print(prob)

prob.solve()

print("-------Solution Status-------")
print("\nStatus:", LpStatus[prob.status])

if LpStatus[prob.status] == 'Optimal':
    print("\n-------Optimal Distribution Plan (Number of Truckloads)-------")

    print("\nFrom Plants to Warehouses:")
    for i in plants:
        for j in warehouses:
            if x[i][j].varValue > 0:
                 print(f"  Plant {i} to Warehouse {j}: {x[i][j].varValue}")


    print("\nFrom Warehouses to Retail Outlets:")
    for j in warehouses:
        for k in retail_outlets:
             if y[j][k].varValue > 0:
                 print(f"  Warehouse {j} to Retail Outlet {k}: {y[j][k].varValue}")

    print("\n-------Minimum Total Shipping Cost-------")
    print(f"Total Cost: ${value(prob.objective):,.2f}") 

else:
    print("\n-------No optimal solution found.-------")

-------Linear Programming Model-------

Maksonsel_Distribution_Problem:
MINIMIZE
425*Flow_P_W_P1_W1 + 560*Flow_P_W_P1_W2 + 510*Flow_P_W_P2_W1 + 600*Flow_P_W_P2_W2 + 470*Flow_W_R_W1_R1 + 505*Flow_W_R_W1_R2 + 490*Flow_W_R_W1_R3 + 390*Flow_W_R_W2_R1 + 410*Flow_W_R_W2_R2 + 440*Flow_W_R_W2_R3 + 0.0
SUBJECT TO
Plant_P1_Output: Flow_P_W_P1_W1 + Flow_P_W_P1_W2 <= 200

Plant_P2_Output: Flow_P_W_P2_W1 + Flow_P_W_P2_W2 <= 300

PP1_WW1_Capacity: Flow_P_W_P1_W1 <= 125

PP1_WW2_Capacity: Flow_P_W_P1_W2 <= 150

PP2_WW1_Capacity: Flow_P_W_P2_W1 <= 175

PP2_WW2_Capacity: Flow_P_W_P2_W2 <= 200

Warehouse_W1_Flow_Conservation: Flow_P_W_P1_W1 + Flow_P_W_P2_W1
 - Flow_W_R_W1_R1 - Flow_W_R_W1_R2 - Flow_W_R_W1_R3 = 0

Warehouse_W2_Flow_Conservation: Flow_P_W_P1_W2 + Flow_P_W_P2_W2
 - Flow_W_R_W2_R1 - Flow_W_R_W2_R2 - Flow_W_R_W2_R3 = 0

WW1_RR1_Capacity: Flow_W_R_W1_R1 <= 100

WW1_RR2_Capacity: Flow_W_R_W1_R2 <= 150

WW1_RR3_Capacity: Flow_W_R_W1_R3 <= 100

WW2_RR1_Capacity: Flow_W_R_W2_R1 <= 125

WW2_RR2_Ca